# Using the Equities India (BSE) data-set

The R version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-R/EquitiesIndiaBse.ipynb)

In [9]:
import os
os.environ['PLUTO_PY_CONFIG_PATH'] = '/usr/share/pluto'

from sqlalchemy import func, and_, or_, text
from plutoPy.model import RoutingSession, EquitiesIndiaBse
from datetime import date, datetime

symbol = "SBIN" #fetch info about State Bank of India

## get BSE's code for a symbol

In [3]:
secInfo = (RoutingSession.session.query(EquitiesIndiaBse.Tickers)
           .filter(EquitiesIndiaBse.Tickers.SYMBOL == symbol)
           .all())

print(secInfo)

code = secInfo[0].CODE

[SBIN(A): State Bank Of India, Banks]


### fetch some "misc" info

In [4]:
end_dt = RoutingSession.session.query(func.max(EquitiesIndiaBse.MiscInfo.TIME_STAMP)).scalar() 

results = (RoutingSession.session.query(EquitiesIndiaBse.MiscInfo)
            .filter(and_(EquitiesIndiaBse.MiscInfo.TIME_STAMP == end_dt, EquitiesIndiaBse.MiscInfo.CODE == code))
            .all())

print("misc info:")
for instance in results:
    print(instance)

misc info:
500112/2019-Sep-27: 107932.02,251004.7,13.68


### fetch the latest end-of-day prices

In [5]:
end_dt = RoutingSession.session.query(func.max(EquitiesIndiaBse.EodTimeSeries.TIME_STAMP)).scalar()

results = (RoutingSession.session.query(EquitiesIndiaBse.EodTimeSeries)
            .filter(and_(EquitiesIndiaBse.EodTimeSeries.TIME_STAMP == end_dt, EquitiesIndiaBse.EodTimeSeries.CODE == code))
            .all())

print(f"fetched: {len(results)}")
for instance in results:
    print(instance)

fetched: 1
500112(A)/2019-Sep-27: 286.05,280.0,283.0,281.25,1793222.0


### fetch the last 10 day EOD prices

In [7]:
results = (RoutingSession.session.query(EquitiesIndiaBse.EodTimeSeries)
            .filter(EquitiesIndiaBse.EodTimeSeries.CODE == code)
            .order_by(EquitiesIndiaBse.EodTimeSeries.TIME_STAMP.desc())
            .limit(10)
            .all())

for instance in results:
    print(instance)   

500112(A)/2019-Sep-27: 286.05,280.0,283.0,281.25,1793222.0
500112(A)/2019-Sep-26: 289.7,277.0,281.6,281.9,3447677.0
500112(A)/2019-Sep-25: 299.4,278.6,299.4,280.15,2970811.0
500112(A)/2019-Sep-24: 315.0,301.0,313.2,302.45,1712432.0
500112(A)/2019-Sep-23: 315.5,300.4,308.2,313.6,3682690.0
500112(A)/2019-Sep-20: 307.6,268.55,274.05,301.7,4417530.0
500112(A)/2019-Sep-19: 280.5,273.2,280.5,274.05,1126938.0
500112(A)/2019-Sep-18: 282.85,274.5,277.0,280.5,1000704.0
500112(A)/2019-Sep-17: 286.35,273.1,285.3,273.9,1084815.0
500112(A)/2019-Sep-16: 290.45,284.05,287.9,284.65,824210.0


### fetch the last 24 quarter EPS

In [10]:
results = (RoutingSession.session.query(EquitiesIndiaBse.CorporateResults)
            .filter(and_(EquitiesIndiaBse.CorporateResults.CODE == code, 
                         EquitiesIndiaBse.CorporateResults.KEY.ilike('%diluted%'),
                         EquitiesIndiaBse.CorporateResults.KEY.ilike('%after%'),
                         func.datediff(text('day'), EquitiesIndiaBse.CorporateResults.PERIOD_BEGIN, EquitiesIndiaBse.CorporateResults.PERIOD_END) < 100))
            .order_by(EquitiesIndiaBse.CorporateResults.PERIOD_END.desc())
            .limit(24)
            .all())

print("the last 24 quarter EPS: ")  
for instance in results:
    print(instance)         

the last 24 quarter EPS: 
2019-Apr-01:2019-Jun-30 EPS after Extraordinary items (in Rs)/Basic & Diluted EPS after Extraordinary items: 3.31
2019-Jan-01:2019-Mar-31 nan/Diluted EPS after Extraordinary items: 0.94
2018-Oct-01:2018-Dec-31 EPS after Extraordinary items (in Rs)/Basic & Diluted EPS after Extraordinary items: 5.28
2018-Jul-01:2018-Sep-30 EPS after Extraordinary items (in Rs)/Basic & Diluted EPS after Extraordinary items: 0.65
2018-Apr-01:2018-Jun-30 EPS after Extraordinary items (in Rs)/Basic & Diluted EPS after Extraordinary items: -4.74
2018-Jan-01:2018-Mar-31 nan/Diluted EPS after Extraordinary items: -8.92
2017-Oct-01:2017-Dec-31 EPS after Extraordinary items (in Rs)/Basic & Diluted EPS after Extraordinary items: -2.19
2017-Jul-01:2017-Sep-30 EPS after Extraordinary items (in Rs)/Basic & Diluted EPS after Extraordinary items: 2.11
2017-Apr-01:2017-Jun-30 EPS after Extraordinary items (in Rs)/Basic & Diluted EPS after Extraordinary items: 3.7
2017-Jan-01:2017-Mar-31 nan/Di

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)